<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#LENET-NETWORK" data-toc-modified-id="LENET-NETWORK-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>LENET NETWORK</a></span></li></ul></div>

In [1]:
import torch
from torchsummary import summary
import torch.nn.functional as F
from torch import nn

## LENET NETWORK
<img src='../images/lenet.jpg'>
<img src='../images/lenet.svg'>

 (source: Dive into Deep Learning by Aston Zhang, Zachary C. Lipton, Mu Li, and Alexander J. Smola page 252-3)
  

To read more on LENET visit :

<a href='http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf'>GradientBased Learning Applied to Document Recognition</a>

In [2]:
class Reshape(nn.Module):
    def forward(self, x):
        return x.view(-1,1,28,28)
lenet=nn.Sequential(
                      Reshape(),
                     nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5,padding=2),nn.Sigmoid(),
                     nn.AvgPool2d(kernel_size=2,stride=2),
                     nn.Conv2d(6,16,kernel_size=5),nn.Sigmoid(),
                     nn.AvgPool2d(kernel_size=2,stride=2),
                     nn.Flatten(),
                     nn.Linear(16*5*5,120),nn.Sigmoid(),
                     nn.Linear(120,84),nn.Sigmoid(),
                     nn.Linear(84,10))

In [3]:
class Lenet(nn.Module):
    def __init__(self):
        super().__init__()
        self.c1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5,padding=2)
        self.avg1=nn.AvgPool2d(kernel_size=2,stride=2)
        self.c2=nn.Conv2d(6,16,kernel_size=5)
        self.avg2=nn.AvgPool2d(kernel_size=2,stride=2)
        self.f=nn.Flatten()
        self.l1=nn.Linear(16*5*5,120)
        self.l2=nn.Linear(120,84)
        self.l3=nn.Linear(84,10)
        self.relu=nn.ReLU()
    def forward(self,x):
        x=x.reshape(-1,1,28,28)
        h1=self.avg1(self.relu(self.c1(x)))
        h2=self.avg2(self.relu(self.c2(h1)))
        h2=self.f(h2)
        h3=self.relu(self.l1(h2))
        h4=self.relu(self.l2(h3))
        output=self.l3(h4)
        return output
lenet1=Lenet()

As compared to the original network, we took the liberty of replacing the Gaussian activation in
the last layer by a regular dense layer, which tends to be significantly more convenient to train.
Other than that, this network matches the historical definition of LeNet5.
Next, let us take a look of an example. As shown in Fig. 6.6.2, we feed a single-channel example
of size 28 × 28 into the network and perform a forward computation layer by layer printing the
output shape at each layer to make sure we understand what is happening here

In [4]:
X = torch.randn(size=(1, 1, 28, 28), dtype=torch.float32)
summary(lenet1,X)

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 6, 28, 28]           156
├─ReLU: 1-2                              [-1, 6, 28, 28]           --
├─AvgPool2d: 1-3                         [-1, 6, 14, 14]           --
├─Conv2d: 1-4                            [-1, 16, 10, 10]          2,416
├─ReLU: 1-5                              [-1, 16, 10, 10]          --
├─AvgPool2d: 1-6                         [-1, 16, 5, 5]            --
├─Flatten: 1-7                           [-1, 400]                 --
├─Linear: 1-8                            [-1, 120]                 48,120
├─ReLU: 1-9                              [-1, 120]                 --
├─Linear: 1-10                           [-1, 84]                  10,164
├─ReLU: 1-11                             [-1, 84]                  --
├─Linear: 1-12                           [-1, 10]                  850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
To

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 6, 28, 28]           156
├─ReLU: 1-2                              [-1, 6, 28, 28]           --
├─AvgPool2d: 1-3                         [-1, 6, 14, 14]           --
├─Conv2d: 1-4                            [-1, 16, 10, 10]          2,416
├─ReLU: 1-5                              [-1, 16, 10, 10]          --
├─AvgPool2d: 1-6                         [-1, 16, 5, 5]            --
├─Flatten: 1-7                           [-1, 400]                 --
├─Linear: 1-8                            [-1, 120]                 48,120
├─ReLU: 1-9                              [-1, 120]                 --
├─Linear: 1-10                           [-1, 84]                  10,164
├─ReLU: 1-11                             [-1, 84]                  --
├─Linear: 1-12                           [-1, 10]                  850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
To

In [5]:
X = torch.randn(size=(1, 1, 28, 28), dtype=torch.float32)
for layer in lenet:
    X = layer(X)
    print(layer.__class__.__name__,'output shape: \t',X.shape)

Reshape output shape: 	 torch.Size([1, 1, 28, 28])
Conv2d output shape: 	 torch.Size([1, 6, 28, 28])
Sigmoid output shape: 	 torch.Size([1, 6, 28, 28])
AvgPool2d output shape: 	 torch.Size([1, 6, 14, 14])
Conv2d output shape: 	 torch.Size([1, 16, 10, 10])
Sigmoid output shape: 	 torch.Size([1, 16, 10, 10])
AvgPool2d output shape: 	 torch.Size([1, 16, 5, 5])
Flatten output shape: 	 torch.Size([1, 400])
Linear output shape: 	 torch.Size([1, 120])
Sigmoid output shape: 	 torch.Size([1, 120])
Linear output shape: 	 torch.Size([1, 84])
Sigmoid output shape: 	 torch.Size([1, 84])
Linear output shape: 	 torch.Size([1, 10])
